In [1]:
import os
import numpy as np


import json
from tqdm import tqdm_notebook

import cv2
import matplotlib.pyplot as plt

from skimage.draw import polygon, polygon_perimeter
from skimage import io

from skimage.morphology import disk, dilation, erosion


def plot(image, title=None, cmap='gray'):
    plt.rcParams['figure.figsize'] = 20, 20
    
    plt.imshow(image, cmap)
    plt.title(title)
    plt.show()

    
def draw_contour(img_shape, x, y, channel):
    # init a mask
    mask = np.zeros(img_shape, dtype=np.uint8)
    x = np.array(x)
    x = [min(i, img_shape[1]) for i in x]
    y = np.array(y)
    y = [min(i, img_shape[0]) for i in y]
    # func polygon returns ndarray of int Pixel coordinates of polygon.
    width, height = max(x)-min(x), max(y)-min(y)
    sq = width*height
    # print(f'Contour width: {width*0.5} pixels')
    # May be used to directly index into an array, e.g. img[rr, cc] = 1.
    mask_x, mask_y = polygon(x, y)
    # fill the contour
    mask[mask_y, mask_x, channel] = 255
    # erose contour
#     kernel = np.ones((1, int(width*0.5)),np.uint8)
#     mask[:, :, channel] = cv2.erode(mask[:, :, channel],kernel,iterations = 1)
    return mask


def draw_contours_above(mask, x, y, channel=3):
    
    selen = disk(2)
    
    x = np.array(x)
    y = np.array(y)
    
    cnt_x, cnt_y = polygon_perimeter(x, y)
    # plot contour
    mask[cnt_y, cnt_x, channel] = 255
    
    mask[:, :, 2] = dilation(mask[:, :, 2], selen)
    return mask

### Make a masks from via json, specify *json_path* to your

In [2]:
json_path = 'via_export_json.json'

with open(json_path, 'r') as f:
    meta = json.load(f)
print(meta)

{'30.png19387241': {'filename': '30.png', 'size': 19387241, 'regions': [{'shape_attributes': {'name': 'polyline', 'all_points_x': [573, 479, 472, 449, 429, 442, 1065, 1062, 1052, 1062, 1075, 1075, 1068, 1062, 1082, 1075, 1038, 1025, 569, 569], 'all_points_y': [3, 1125, 1420, 2562, 3108, 3285, 3285, 2649, 2552, 2341, 2150, 2003, 1587, 1296, 864, 596, 20, 3, 3, 104]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polyline', 'all_points_x': [4605, 4561, 4146, 3791, 3560, 3379, 3379, 3439, 3540, 3603, 3503, 3423, 3570, 3818, 3948, 4390, 4581, 4605, 4601, 4390, 4605, 4601, 4494, 4544, 4605, 4601], 'all_points_y': [1949, 1845, 1413, 1098, 998, 968, 924, 857, 894, 861, 770, 703, 640, 844, 924, 1222, 1380, 1396, 1497, 1329, 1510, 1658, 1584, 1688, 1745, 1942]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polyline', 'all_points_x': [3372, 3369, 3369, 3098, 2739, 2555, 2421, 2254, 2170, 2120, 2127, 2147, 2257, 2388, 2639, 2806, 2944, 3212, 3392, 3556, 3479, 3252, 3101, 28

In [3]:
masks = []
filenames = []
missed = []

for file in tqdm_notebook(meta):
    try:
        filename = file.split('.png')[0] + '.png'
        print(filename)
        img = cv2.imread('images/'+filename)
        # get image shape
        img_height, img_weight = img.shape[:2]
        # init a mask
        mask = np.zeros((img_height, img_weight, 3), dtype=np.uint8)
        # iter through marked contours in file
        file_contours = meta[file]['regions']
        for cnt_dict  in file_contours:
            cnt_x, cnt_y = cnt_dict['shape_attributes']['all_points_x'], cnt_dict['shape_attributes']['all_points_y']

            # start with a mask
            channel = 0
            mask += draw_contour(mask.shape, cnt_x, cnt_y, channel)
            
        if mask.sum() > 0:
            masks.append(mask)
            filenames.append(filename)
            io.imsave('masks/'+filename, mask)
        else:
            print(filename)
            missed.append(filename)
            #os.remove('grausy_polushka/all_masks_with_cnts/'+filename)

        
                
    except Exception as e:
        missed.append(filename)
        print(f'ERROR: {e}', filename)
        #os.remove('grausy_polushka/all_marked_files/'+filename)

/media/ansary/DriveData/UPWORK/WireDetection/pyWireDetect/wirevenv/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


30.png
29.png
28.png
27.png
26.png
25.png
24.png
23.png
22.png
21.png
20.png
19.png
ERROR: 'NoneType' object has no attribute 'shape' 19.png
18.png
17.png
16.png
15.png
14.png
13.png
12.png
11.png
10.png
9.png
8.png
7.png
6.png
5.png
4.png
3.png
2.png
1.png



In [26]:
plot(masks[5])

IndexError: list index out of range

#### Drop unplotted images if exist

In [0]:
masks = sorted([i for i in os.listdir('masks/') if i.endswith('.jpg')])

images = sorted([i for i in os.listdir('images/') if i.endswith('.jpg')])
for i in images:
    if i not in masks:
        print(f'File to drop: {i}')
        os.remove(f'images/{i}')

NameError: name 'os' is not defined

### Prepare dataset, divide to train, val, test

#### Create the following structure:
        
    Data_directory/
    |
    |
    ├── _train/
    │   ├── images/
    │   │   ├── images/
    |           ├── 1.jpg
    |           ├── 2.jpg
    |           ├── ...
    │   ├── masks/
    |   |   ├── masks/
    |           ├── 1.jpg
    |           ├── 2.jpg
    |           ├── ...
    ├── _val/
    |   ├── _images/
    |   |   ├── images/
    |           ├── 1.jpg
    |           ├── 2.jpg
    |           ├── ...
    |   ├── masks/
    |   |   ├── masks/
    |           ├── 1.jpg
    |           ├── 2.jpg
    |           ├── ...
    ├── _test/
    |   ├── _images/
    |   |   ├── images/
    |           ├── 1.jpg
    |           ├── 2.jpg
    |           ├── ...
    |   ├── masks/
    |   |   ├── masks/
    |           ├── 1.jpg
    |           ├── 2.jpg
    |           ├── ...

In [0]:
import shutil
from random import shuffle

In [0]:
names = images
shuffle(names)

print(len(names))

101


In [0]:
train = names[:70]
val = names[70:90]
test = names[90:]

print('Train: ', len(train))
print('Test: ', len(test))
print('Val: ', len(val))

Train:  70
Test:  11
Val:  20


#### Copy files from *iamges/* and *masks/* folders

In [0]:
for name in train:
    try:
        shutil.copy('images/'+name, 'wire_set/train/images/images/'+name)
        shutil.copy('masks/'+name, 'wire_set/train/masks/masks/'+name)
    except FileNotFoundError as e:
        print('Train', e)
        
for name in test:
    try:
        shutil.copy('images/'+name, 'wire_set/test/images/images/'+name)
        shutil.copy('masks/'+name, 'wire_set/test/masks/masks/'+name)
    except FileNotFoundError as e:
        print('Test', e)
        
for name in val:
    try:
        shutil.copy('images/'+name, 'wire_set/val/images/images/'+name)
        shutil.copy('masks/'+name, 'wire_set/val/masks/masks/'+name)
    except FileNotFoundError as e:
        print('Val', e)